In [1]:
TRIPS_PATH = 'https://raw.githubusercontent.com/IsamAljawarneh/datasets/1c2a6af7dea7aa93105ac1d1d0118d07bd681d8a/data/NYC_Pilot2_PM_Part1.csv'
POLYGON_PATH = 'https://raw.githubusercontent.com/IsamAljawarneh/datasets/1c2a6af7dea7aa93105ac1d1d0118d07bd681d8a/data/nyc_polygon.geojson'

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import MultiPolygon

In [3]:
trips = pd.read_csv(TRIPS_PATH)

In [4]:
gdf_trips = gpd.GeoDataFrame(trips, geometry=gpd.points_from_xy(trips.longitude, trips.latitude))
gdf_trips.set_crs('epsg:4326', inplace=True)

,SensorID,time,latitude,longitude,bin0,bin1,bin2,bin3,bin4,bin5,...,bin18,bin19,bin20,bin21,bin22,bin23,temperature,humidity,pm25,geometry
0,NYCP2_CS01A,1631277304,40.847672,-73.869316,11,1,1,0,0,0,...,0,0,0,0,0,0,23.7,57.3,4.508813,POINT (-73.86932 40.84767)
1,NYCP2_CS01A,1631277308,40.847668,-73.869316,22,4,1,0,0,2,...,0,0,0,0,0,0,23.7,57.8,5.462420,POINT (-73.86932 40.84767)
2,NYCP2_CS01A,1631277313,40.847649,-73.869362,40,1,1,0,0,1,...,0,0,0,0,0,0,23.7,57.8,5.154881,POINT (-73.86936 40.84765)
3,NYCP2_CS01A,1631277318,40.847649,-73.869362,26,1,0,0,0,0,...,0,0,0,0,0,0,23.6,57.6,4.508813,POINT (-73.86936 40.84765)
4,NYCP2_CS01A,1631277323,40.847649,-73.869362,44,4,0,1,0,0,...,0,0,0,0,0,0,23.6,57.5,5.539503,POINT (-73.86936 40.84765)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169994,NYCP2_CS03A,1631457109,40.823353,-73.890488,115,11,2,0,1,0,...,0,0,0,0,0,0,24.6,54.8,5.460360,POINT (-73.89049 40.82335)
169995,NYCP2_CS03A,1631457114,40.823349,-73.890480,132,8,2,0,0,0,...,0,0,0,0,0,0,24.6,54.8,5.298209,POINT (-73.89048 40.82335)
169996,NYCP2_CS03A,1631457119,40.823349,-73.890480,147,14,0,0,0,0,...,0,0,0,0,0,0,24.6,54.8,6.470661,POINT (-73.89048 40.82335)
169997,NYCP2_CS03A,1631457124,40.823345,-73.890488,121,8,2,0,1,1,...,0,0,0,0,0,0,24.6,54.6,6.424142,POINT (-73.89049 40.82335)


In [5]:
geojson_file = POLYGON_PATH
neighborhoods_original = gpd.read_file(geojson_file)

In [6]:
sjoined_trips_original = gpd.sjoin(gdf_trips, neighborhoods_original)
sjoined_trips_original.head(5)

,SensorID,time,latitude,longitude,bin0,bin1,bin2,bin3,bin4,bin5,...,bin23,temperature,humidity,pm25,geometry,index_right,neighborhood,boroughCode,borough,@id
0,NYCP2_CS01A,1631277304,40.847672,-73.869316,11,1,1,0,0,0,...,0,23.7,57.3,4.508813,POINT (-73.86932 40.84767),38,Bronx Park,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
1,NYCP2_CS01A,1631277308,40.847668,-73.869316,22,4,1,0,0,2,...,0,23.7,57.8,5.462420,POINT (-73.86932 40.84767),38,Bronx Park,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
2,NYCP2_CS01A,1631277313,40.847649,-73.869362,40,1,1,0,0,1,...,0,23.7,57.8,5.154881,POINT (-73.86936 40.84765),38,Bronx Park,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
3,NYCP2_CS01A,1631277318,40.847649,-73.869362,26,1,0,0,0,0,...,0,23.6,57.6,4.508813,POINT (-73.86936 40.84765),38,Bronx Park,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
4,NYCP2_CS01A,1631277323,40.847649,-73.869362,44,4,0,1,0,0,...,0,23.6,57.5,5.539503,POINT (-73.86936 40.84765),38,Bronx Park,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...


In [7]:
df_pm25 = (sjoined_trips_original
                          .groupby('neighborhood')
                          .pm25
                          .agg(list)
                          .to_frame("pm25_list")
                          .reset_index())# Let's count each points inside the hexagon

df_pm25['average_pm25_value'] =(df_pm25['pm25_list'].apply(lambda x:sum(x)/len(x)))
df_pm25.drop('pm25_list',axis = 1, inplace=True)

In [8]:
df_pm25.head()

,neighborhood,average_pm25_value
0,Allerton,2.940433
1,Baychester,3.464289
2,Belmont,4.138367
3,Bronx Park,4.141387
4,Bronxdale,3.787668


In [9]:
df_pm25 = pd.merge(df_pm25, neighborhoods_original, on='neighborhood')

In [10]:
df_pm25.head()

,neighborhood,average_pm25_value,boroughCode,borough,@id,geometry
0,Allerton,2.940433,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-73.84860 40.87167, -73.84582 40.870..."
1,Baychester,3.464289,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-73.83111 40.86555, -73.82930 40.862..."
2,Belmont,4.138367,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-73.88067 40.85716, -73.88074 40.856..."
3,Bronx Park,4.141387,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-73.86995 40.85589, -73.86861 40.852..."
4,Bronxdale,3.787668,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-73.86888 40.85722, -73.85639 40.857..."


In [11]:
print(len(df_pm25["neighborhood"]), df_pm25["neighborhood"].nunique())

50 48


In [12]:
def polygonToMultiPolygon(gdf):
    grouped = gdf.groupby('neighborhood').geometry.agg(lambda x: MultiPolygon(x.tolist()) if len(x) > 1 else x.iloc[0])

    # Reset index to convert the grouped Series back to a DataFrame
    combined_gdf = grouped.reset_index()

    # Merge the aggregated geometries back with the original GeoDataFrame to retain other columns
    merged_gdf = combined_gdf.merge(gdf.drop('geometry', axis=1), on='neighborhood').set_geometry('geometry')
    merged_gdf.drop_duplicates(inplace=True)

    # Now merged_gdf contains a single row for each neighborhood with a MultiPolygon geometry
    return merged_gdf

In [13]:
df_pm25_merged = polygonToMultiPolygon(df_pm25)

In [14]:
print(len(df_pm25_merged['neighborhood']), df_pm25_merged['neighborhood'].nunique())

48 48


In [15]:
df_pm25_merged

,neighborhood,geometry,average_pm25_value,boroughCode,borough,@id
0,Allerton,"POLYGON ((-73.84860 40.87167, -73.84582 40.870...",2.940433,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
1,Baychester,"POLYGON ((-73.83111 40.86555, -73.82930 40.862...",3.464289,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
2,Belmont,"POLYGON ((-73.88067 40.85716, -73.88074 40.856...",4.138367,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
3,Bronx Park,"POLYGON ((-73.86995 40.85589, -73.86861 40.852...",4.141387,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
4,Bronxdale,"POLYGON ((-73.86888 40.85722, -73.85639 40.857...",3.787668,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
5,Castle Hill,"POLYGON ((-73.85532 40.81441, -73.85538 40.814...",6.046787,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
6,Clason Point,"POLYGON ((-73.85538 40.81429, -73.85532 40.814...",5.116131,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
7,Co-op City,"POLYGON ((-73.82031 40.88552, -73.82008 40.883...",4.761496,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
8,Concourse,"POLYGON ((-73.91533 40.83696, -73.91533 40.836...",3.369017,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...
9,Concourse Village,"POLYGON ((-73.90707 40.83344, -73.90896 40.830...",3.119020,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...


In [16]:
df_pm25_merged.to_csv('../../data/Average pm25 per neighborhood BASELINE.csv')